## Data Collection

### Get US DOT Flight Delay Dataset

In [ ]:
import kagglehub
path = kagglehub.dataset_download("usdot/flight-delays")

In [ ]:
import cudf

airlines_df = cudf.read_csv(path + "/airlines.csv")
airports_df = cudf.read_csv(path + "/airports.csv")
flights_df = cudf.read_csv(path + "/flights.csv")

In [ ]:
flights_df.head()

### Get NOAA Weather Station Data

In [ ]:
import requests
import gzip
import shutil

# URL of the file
url = "https://www.ncei.noaa.gov/pub/data/ghcn/daily/ghcnd-stations.csv"
file = "ghcnd-stations.csv"

# Download the file
response = requests.get(url, stream=True)
with open(file, "wb") as f:
    shutil.copyfileobj(response.raw, f)

In [ ]:
'''
FORMAT OF "ghcnd-stations.txt"

------------------------------
Variable   Columns   Type
------------------------------
ID            1-11   Character
LATITUDE     13-20   Real
LONGITUDE    22-30   Real
ELEVATION    32-37   Real
STATE        39-40   Character
NAME         42-71   Character
GSN FLAG     73-75   Character
HCN/CRN FLAG 77-79   Character
WMO ID       81-85   Character
------------------------------
'''
ghcnd_stations_df = cudf.read_csv(
    file,
    header=None,
    names=["ID", "LATITUDE", "LONGITUDE", "ELEVATION"],
    usecols=[0, 1, 2, 3],  # Select only the columns we need
    skipinitialspace=True
)
ghcnd_stations_df.head()


### Add in NOAA Weather Station IDs for airports

I have a DataFrame, `ghcnd_stations_df` that has a list of weather data stations with their lat & long stored as columns (`LATITUDE`, `LONGITUDE`). I want to use the `airports_df` which also has `LATITUDE` and `LONGITUDE` columns to find the nearest weather station to each airport. Append the `ghcnd_stations_df` `ID` column to `airports_df` as `NOAA_STATION_ID`.

In [ ]:
import cupy as cp
from cuml.neighbors import NearestNeighbors

# Remove rows with NaN lat/lon in either DataFrame
airports_df = airports_df.dropna(subset=['LATITUDE', 'LONGITUDE'])
ghcnd_stations_df = ghcnd_stations_df.dropna(subset=['LATITUDE', 'LONGITUDE'])

# Convert airport/station lat-lon to radians using CuPy
airports_df['lat_rad'] = cp.radians(airports_df['LATITUDE'].to_cupy())
airports_df['lon_rad'] = cp.radians(airports_df['LONGITUDE'].to_cupy())
ghcnd_stations_df['lat_rad'] = cp.radians(ghcnd_stations_df['LATITUDE'].to_cupy())
ghcnd_stations_df['lon_rad'] = cp.radians(ghcnd_stations_df['LONGITUDE'].to_cupy())

# Build the NearestNeighbors model from cuML
stations_coords = ghcnd_stations_df[['lat_rad','lon_rad']].to_cupy()
nn = NearestNeighbors(n_neighbors=1, metric='haversine')
nn.fit(stations_coords)

# Query nearest station for each airport
airports_coords = airports_df[['lat_rad','lon_rad']].to_cupy()
distances, indices = nn.kneighbors(airports_coords)

# Append the station ID to airports_df
airports_df['NOAA_STATION_ID'] = ghcnd_stations_df['ID'].iloc[indices.flatten().get()].to_numpy()

# Remove the temporary columns used for calculations
airports_df.drop(columns=['lat_rad', 'lon_rad'], inplace=True)

In [ ]:
airports_df.head()

## Data Cleanup

Merges the 3 DataFrames together, first by removing their file name prefixes from the column names. Then, by adding in airport information for both the origin and destination airport (merges the Airports DF twice technically), and then joins the airlines table with the flights table. Lastly, we create a datetime object from the existing date columns

In [ ]:
# Remove file name prefix from column names
airlines_df.columns = airlines_df.columns.str.replace(r'^airlines\.csv/', '', regex=True)
airlines_df.rename(columns={'AIRLINE': 'AIRLINE NAME'}, inplace=True)

airports_df.columns = airports_df.columns.str.replace(r'^airports\.csv/', '', regex=True)

flights_df.columns = flights_df.columns.str.replace(r'^flights\.csv/', '', regex=True)

In [ ]:
# Join airlines data to flights table
flights_df = flights_df.merge(airlines_df, left_on='AIRLINE', right_on='IATA_CODE', how='left')
# Remove duplicate IATA_CODE column if it exists
flights_df = flights_df.drop(columns=['IATA_CODE'], errors='ignore')

In [ ]:
# Join airports data to flights table
origin_airports = airports_df.add_prefix('origin_airport/')
destination_airports = airports_df.add_prefix('destination_airport/')

# Rename the index in origin_airports to match the column in flights_df
origin_airports = origin_airports.rename(columns={'origin_airport/IATA_CODE': 'ORIGIN_AIRPORT'})
flights_df = flights_df.merge(origin_airports, on='ORIGIN_AIRPORT', how='left')


# Rename the index in destination_airports to match the column in flights_df
destination_airports = destination_airports.rename(columns={'destination_airport/IATA_CODE': 'DESTINATION_AIRPORT'})
flights_df = flights_df.merge(destination_airports, on='DESTINATION_AIRPORT', how='left')

In [ ]:
# Rename the columns to match the expected format
flights_df.rename(
    columns={
        "YEAR": "year",
        "MONTH": "month",
        "DAY": "day",
    },
    inplace=True,
)

# Try creating the 'DATE' column again
flights_df['DATE'] = cudf.to_datetime(flights_df[['year', 'month', 'day']])

# Optionally, rename columns back to original names
flights_df.rename(
    columns={
        "year": "YEAR",
        "month": "MONTH",
        "day": "DAY",
    },
    inplace=True,
)

In [ ]:
# Replace the values in the CANCELLATION_REASON column
flights_df['CANCELLATION_REASON'] = flights_df['CANCELLATION_REASON'].replace({
    'A': 'Airline/Carrier',
    'B': 'Weather',
    'C': 'National Air System',
    'D': 'Security'
})

### Get Weather Data

In [ ]:
import requests
import gzip
import shutil

# URL of the file
url = "https://www.ncei.noaa.gov/pub/data/ghcn/daily/by_year/2015.csv.gz"
output_gz_file = "2015.csv.gz"
output_csv_file = "2015.csv"

# Download the file
response = requests.get(url, stream=True)
with open(output_gz_file, "wb") as f:
    shutil.copyfileobj(response.raw, f)

# Extract the gzip file
with gzip.open(output_gz_file, "rb") as f_in:
    with open(output_csv_file, "wb") as f_out:
        shutil.copyfileobj(f_in, f_out)

print(f"File downloaded and extracted to {output_csv_file}")

In [ ]:
'''
ID = 11 character station identification code
YEAR/MONTH/DAY = 8 character date in YYYYMMDD format (e.g. 19860529 = May 29, 1986)
ELEMENT = 4 character indicator of element type
DATA VALUE = 5 character data value for ELEMENT
M-FLAG = 1 character Measurement Flag
Q-FLAG = 1 character Quality Flag
S-FLAG = 1 character Source Flag
OBS-TIME = 4-character time of observation in hour-minute format (i.e. 0700 =7:00 am)
'''
weather_df = cudf.read_csv(output_csv_file,
                         header=None,
                         names=['ID', 'YEAR/MONTH/DAY',
                                'ELEMENT', 'DATA_VALUE',
                                'M_FLAG', 'Q_FLAG', 'S_FLAG',
                                'OBS_TIME'])

In [ ]:
# Convert YEAR/MONTH/DAY column to Date type
weather_df['DATE'] = cudf.to_datetime(weather_df['YEAR/MONTH/DAY'], format='%Y%m%d')
weather_df.head()

### Add snowfall data to flight data

In [ ]:
# Filter daily snowfall records
snow_df = weather_df[weather_df['ELEMENT'] == 'SNOW'].copy()
# Convert YEAR/MONTH/DAY column in snow_df to cudf.Timestamp, and rename it to SNOW_DATE
# Ensure SNOW_DATE is of type cudf.Timestamp
snow_df['SNOW_DATE'] = cudf.to_datetime(snow_df['YEAR/MONTH/DAY'].astype(str))

# Convert 'DATE' column in flights_df to have the same dtype as 'SNOW_DATE'
flights_df['DATE'] = flights_df['DATE'].astype(snow_df['SNOW_DATE'].dtype)

# Merge on DATE_ONLY to match daily snowfall
flights_df = flights_df.merge(
    snow_df[['ID', 'SNOW_DATE', 'DATA_VALUE']], # Use DATA_VALUE instead of DAILY_SNOWFALL
    left_on=['origin_airport/NOAA_STATION_ID', 'DATE'],
    right_on=['ID', 'SNOW_DATE'],
    how='left'
)

# Clean up columns no longer needed
flights_df.drop(columns=['ID', 'SNOW_DATE'], inplace=True)

# Rename DATA_VALUE to DAILY_SNOWFALL for clarity
flights_df.rename(columns={'DATA_VALUE': 'DAILY_SNOWFALL'}, inplace=True)

### Export Cleaned Data

In [ ]:
del airlines_df
del airports_df
del origin_airports
del destination_airports
del ghcnd_stations_df
del weather_df
del snow_df

In [ ]:
# Export cleaned data to Parquet file
flights_df.to_parquet("cleaned_flights.parquet", index=False)